In [1]:
import numpy as np
from aocd import get_data

# Part 1
Add test case

In [2]:
test_str = r".|...\....\n|.-.\.....\n.....|-...\n........|.\n..........\n.........\\n..../.\\..\n.-.-/..|..\n.|....-|.\\n..//.|...."
test = test_str.split(r"\n")
print(test)

['.|...\\....', '|.-.\\.....', '.....|-...', '........|.', '..........', '.........\\', '..../.\\\\..', '.-.-/..|..', '.|....-|.\\', '..//.|....']


Add actual puzzle input

In [3]:
data_str = get_data(day=16)
data = data_str.split("\n")
data[0]

'\\\\.........../..............-..../....\\.......\\.................|....|..........|............-\\.....|.........'

Create empty list of beams to store active beams in

In [4]:
beams = []
data_energized = np.empty((len(test),len(test[0])), dtype=object)
for i in range(data_energized.shape[0]):
    for j in range(data_energized.shape[1]):
        data_energized[i,j] = []

Create beam class

In [63]:
class Beam:
    def __init__(self, row = 0, i = 0, direction="r", delete=False, first=False):
        global data
        self.row = row
        self.i = i
        self.dir = direction
        if first:
            if self.dir == "d" and data[self.row][self.i] == "/":
                self.mirror("l")
            elif self.dir == "d" and data[self.row][self.i] == "\\":
                self.mirror("r")
            elif self.dir == "d" and data[self.row][self.i] == "-":
                self.split("l")
            elif self.dir == "u" and data[self.row][self.i] == "/":
                self.mirror("r")
            elif self.dir == "u" and data[self.row][self.i] == "\\":
                self.mirror("l")
            elif self.dir == "u" and data[self.row][self.i] == "-":
                self.split("l")
            elif self.dir == "l" and data[self.row][self.i] == "/":
                self.mirror("d")
            elif self.dir == "l" and data[self.row][self.i] == "\\":
                self.mirror("u")
            elif self.dir == "l" and data[self.row][self.i] == "|":
                self.split("u")
            elif self.dir == "r" and data[self.row][self.i] == "/":
                self.mirror("u")
            elif self.dir == "r" and data[self.row][self.i] == "\\":
                self.mirror("d")
            elif self.dir == "r" and data[self.row][self.i] == "|":
                self.split("u")
        self.delete = False
        beams.append(self)
        
    def split(self, direction):
        self.dir = direction
        if direction == "l":
            split_dir = "r"
        elif direction == "r":
            split_dir = "l"
        elif direction == "u":
            split_dir = "d"
        elif direction == "d":
            split_dir = "u"
        Beam(row=self.row, i=self.i, direction=split_dir)
        
    def mirror(self, direction):
        self.dir = direction
        
    def energize(self):
        global data_energized
        if self.dir not in data_energized[self.row][self.i]:
            data_energized[self.row, self.i].append(self.dir)
        else:
            self.delete = True
        
    def move(self, contraption=test):
        # complete directional move
        if self.dir == "r":
            self.i += 1
        elif self.dir == "l":
            self.i -= 1
        elif self.dir == "u":
            self.row -= 1
        elif self.dir == "d":
            self.row += 1
        
        # check if on contraption
        if self.row >= len(contraption) or self.i >= len(contraption[0]) or self.row == -1 or self.i == -1:
            self.delete=True
        
        # post move admin
        if not self.delete:
            # add direction to data_energized
            self.energize()
                
            # check if on mirror or splitter
            new_tile = contraption[self.row][self.i]
            if self.dir in ["r","l"] and new_tile == "|":
                self.split("u")
            elif self.dir in ["u","d"] and new_tile == "-":
                self.split("l")
            elif self.dir == "u" and new_tile == "\\":
                self.mirror("l")
            elif self.dir == "u" and new_tile == "/":
                self.mirror("r")
            elif self.dir == "d" and new_tile == "\\":
                self.mirror("r")
            elif self.dir == "d" and new_tile == "/":
                self.mirror("l")
            elif self.dir == "r" and new_tile == "\\":
                self.mirror("d")
            elif self.dir == "r" and new_tile == "/":
                self.mirror("u")
            elif self.dir == "l" and new_tile == "\\":
                self.mirror("u")
            elif self.dir == "l" and new_tile == "/":
                self.mirror("d")
            
            

Create functions to reset while debugging

In [74]:
def reset_puzzle(puzzle_type="test"):
    global beams,data_energized
    beams = []
    if puzzle_type == "test":
        data_str = r".|...\....\n|.-.\.....\n.....|-...\n........|.\n..........\n.........\\n..../.\\..\n.-.-/..|..\n.|....-|.\\n..//.|...."
        data = data_str.split(r"\n")
    elif puzzle_type == "actual":
        data_str = get_data(day=16)
        data = data_str.split("\n")
    data_energized = np.empty((len(data),len(data[0])), dtype=object)
    for i in range(data_energized.shape[0]):
        for j in range(data_energized.shape[1]):
            data_energized[i,j] = []
    return data

In [75]:
def status(data=test,print_results=True):
    global data_energized
    non_empty_count = np.sum([1 for row in data_energized for item in row if len(item) > 0])
    if print_results:
        print("\n=====Contraption Energized=====")
        print("   0123456789")
        for i,val in enumerate(data):
            print_str = f"{i}: "
            for j, val2 in enumerate(val):
                if data_energized[i,j] == []:
                    print_str += val2
                elif len(data_energized[i,j]) > 1:
                    print_str += str(len(data_energized[i,j]))
                else:
                    print_str += data_energized[i,j][0]
            print(print_str)    
        print(f"There are {non_empty_count} tiles energized")
        if len([beam for beam in beams if not beam.delete]) > 0:
            print("\n==========Beams==========")
            for i, beam in enumerate(beams):
                if not beam.delete:
                    print(f"{i}: {beam.row=} {beam.i=} {beam.dir=}")
    return non_empty_count

In [76]:
data = reset_puzzle()
Beam(first=True)
beams[0].energize()
status()


=====Contraption Energized=====
   0123456789
0: r|...\....
1: |.-.\.....
2: .....|-...
3: ........|.
4: ..........
5: .........\
6: ..../.\\..
7: .-.-/..|..
8: .|....-|.\
9: ..//.|....
There are 1 tiles energized

==========Beams==========
0: beam.row=0 beam.i=0 beam.dir='r'


1

In [77]:
for beam in beams:
    while not beam.delete:
        beam.move()
        
status()


=====Contraption Energized=====
   0123456789
0: r2lllu....
1: |d-.\u....
2: .d...lurrr
3: .d...du.|.
4: .d...du...
5: .d...du..\
6: .d..u22u..
7: l2rrrddu..
8: .llll2dr.\
9: .d//.d.d..
There are 46 tiles energized


46

In [78]:
data = reset_puzzle("actual")
Beam(first=True)
beams[0].energize()

for beam in beams:
    while not beam.delete:
        beam.move(contraption=data)
        
status(data=data,print_results=False)

8034

# Part 2

In [84]:
data = reset_puzzle("actual")
max_energized = 0
for direction in ["l","r","u","d"]:
    for i in range(len(data)):
        data = reset_puzzle("actual")
        if direction == "r":
            Beam(row=i,direction=direction,first=True)
        elif direction == "l":
            Beam(row=i,i=len(data)-1,direction=direction,first=True)
        elif direction == "u":
            Beam(row=len(data)-1,i=i,direction=direction,first=True)
        else:
            Beam(i=i,direction=direction,first=True)
            
        beams[0].energize()
        
        for beam in beams:
            while not beam.delete:
                beam.move(contraption=data)
                
        energized_count = status(data=data,print_results=False)
        if energized_count > max_energized:
            max_energized = energized_count
        
    print("finished",direction)

max_energized

finished l
finished r
finished u
finished d


8225

In [69]:
len(data)

110